## Extra process on dataset

- Find new images when a new dataset is released

In [2]:
import numpy as np
import pandas as pd
from glob import glob
import os
import sys

sys.path.insert(0, os.path.abspath('../../utils/'))
import data_utils as du

In [3]:
path = '../../dataset/unsplash-dataset-v1.1.0/'
documents = ['photos']

datasets = {}

for doc in documents:
    files = glob(path + doc + ".tsv*")

    subsets = []
    for filename in files:
        df = pd.read_csv(filename, sep='\t', header=0)
        subsets.append(df)

    datasets[doc] = pd.concat(subsets, axis=0, ignore_index=True)

##  Unsplash v1.1.0+

#### Find new photos when a new version is released
The logic is to iterate over the folder with saved images, and find those from the new dataset, that their path is missing

In [4]:
df = datasets.copy()
photos = df['photos'].loc[:,['photo_id','photo_url', 'photo_image_url','exif_camera_make','exif_camera_model','exif_iso','exif_focal_length','exif_aperture_value','exif_exposure_time','photo_width', 'photo_height', 'photo_aspect_ratio']]

In [5]:
im_path = '/media/steve/Data2/unsplash-dataset/unsplashed-full/'
remained_ids = []
for i, p in enumerate(photos['photo_id']):
    if not os.path.exists(im_path+p+'.jpg'):
        remained_ids.append(photos.loc[photos['photo_id']==p].index.values)

KeyboardInterrupt: 

In [ ]:
new_photos = photos.take(np.array(remained_ids).ravel())
new_photos.shape

In [19]:
new_photos.reset_index(drop=True,inplace=True)

### Clean new dataset

In [20]:
du.clean_iso(new_photos)
du.clean_focal(new_photos,'mm|-|~')
du.clean_exposures(new_photos,'s')
du.clean_apertures(new_photos,',|f/|undef|inf|Inf|f')

### Save new dataset

It can now be loaded in dataset_exploration

In [ ]:
new_photos.to_csv('../../dataset/new_photos.csv')